## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khatanga,RU,71.9667,102.5000,45.43,72,99,10.60,overcast clouds
1,1,Kon Tum,VN,14.3500,108.0000,71.40,99,100,2.37,light rain
2,2,Sao Filipe,CV,14.8961,-24.4956,76.87,78,100,7.87,overcast clouds
3,3,Sawtell,AU,-30.3833,153.1000,60.82,67,75,12.66,broken clouds
4,4,Dharchula,IN,29.8500,80.5333,72.43,91,99,2.42,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 85


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
17,17,Martinsburg,US,39.4562,-77.9639,83.30,73,100,5.73,overcast clouds
23,23,Butaritari,KI,3.0707,172.7902,81.64,71,30,16.46,scattered clouds
27,27,Rasht,IR,37.2808,49.5832,80.42,78,75,4.61,broken clouds
31,31,Xiamen,CN,24.4798,118.0819,81.99,86,59,5.88,broken clouds
35,35,Karapyshi,UA,49.6424,30.7900,83.91,25,43,13.94,scattered clouds
38,38,Ponta Do Sol,PT,32.6667,-17.1000,84.70,65,0,3.33,clear sky
41,41,Shimoda,JP,34.6667,138.9500,80.10,91,59,11.32,broken clouds
50,50,Yanam,IN,16.7333,82.2167,82.60,79,99,4.74,overcast clouds
52,52,Lewiston,US,44.1003,-70.2148,83.71,55,20,6.91,few clouds
53,53,Puyang,CN,29.4603,119.8861,81.36,81,58,0.43,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                97
City                   97
Country                97
Lat                    97
Lng                    97
Max Temp               97
Humidity               97
Cloudiness             97
Wind Speed             97
Current Description    97
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.dropna(how='any', inplace=True)
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
17,17,Martinsburg,US,39.4562,-77.9639,83.30,73,100,5.73,overcast clouds
23,23,Butaritari,KI,3.0707,172.7902,81.64,71,30,16.46,scattered clouds
27,27,Rasht,IR,37.2808,49.5832,80.42,78,75,4.61,broken clouds
31,31,Xiamen,CN,24.4798,118.0819,81.99,86,59,5.88,broken clouds
35,35,Karapyshi,UA,49.6424,30.7900,83.91,25,43,13.94,scattered clouds
38,38,Ponta Do Sol,PT,32.6667,-17.1000,84.70,65,0,3.33,clear sky
41,41,Shimoda,JP,34.6667,138.9500,80.10,91,59,11.32,broken clouds
50,50,Yanam,IN,16.7333,82.2167,82.60,79,99,4.74,overcast clouds
52,52,Lewiston,US,44.1003,-70.2148,83.71,55,20,6.91,few clouds
53,53,Puyang,CN,29.4603,119.8861,81.36,81,58,0.43,broken clouds


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Martinsburg,US,83.30,overcast clouds,39.4562,-77.9639,
23,Butaritari,KI,81.64,scattered clouds,3.0707,172.7902,
27,Rasht,IR,80.42,broken clouds,37.2808,49.5832,
31,Xiamen,CN,81.99,broken clouds,24.4798,118.0819,
35,Karapyshi,UA,83.91,scattered clouds,49.6424,30.7900,
38,Ponta Do Sol,PT,84.70,clear sky,32.6667,-17.1000,
41,Shimoda,JP,80.10,broken clouds,34.6667,138.9500,
50,Yanam,IN,82.60,overcast clouds,16.7333,82.2167,
52,Lewiston,US,83.71,few clouds,44.1003,-70.2148,
53,Puyang,CN,81.36,broken clouds,29.4603,119.8861,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
  
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Martinsburg,US,83.30,overcast clouds,39.4562,-77.9639,"Holiday Inn Martinsburg, an IHG Hotel"
23,Butaritari,KI,81.64,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
27,Rasht,IR,80.42,broken clouds,37.2808,49.5832,Travellers Camping
31,Xiamen,CN,81.99,broken clouds,24.4798,118.0819,Xiamen Dongchen Hotel
35,Karapyshi,UA,83.91,scattered clouds,49.6424,30.7900,Zolotyy Kolos
...,...,...,...,...,...,...,...
680,Manggar,ID,81.03,overcast clouds,-2.8833,108.2667,Guest Hotel
689,Sulangan,PH,83.48,overcast clouds,11.1373,123.7228,Maia's Beach Resort
690,Kumo,NG,84.51,broken clouds,10.0447,11.2122,Ajegunle Lodge(corpers Lodge)
694,Tahoua,NE,81.97,broken clouds,14.8888,5.2692,Bungalows Guesthouse


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))